# Algoritmo Genético

In [ ]:
import timeit
import numpy as np
import pandas as pd
from scripts.queso_model import model_vars, model_data, objective_func, balance, alloc_df
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.sampling import Sampling
from pymoo.core.crossover import Crossover
from pymoo.core.mutation import Mutation
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.termination.default import DefaultSingleObjectiveTermination
from pymoo.optimize import minimize

#Escenario 1
folder = './data/escenario_1/' # Carpeta con los datos
demanda = 60 # Demanda del cliente
ctiempo = 1000 # Costo por unidad de tiempo

#Escenario 2
#folder = './data/escenario_2/' # Carpeta con los datos
#demanda = 12000 # Demanda del cliente
#ctiempo = 1000 # Costo por unidad de tiempo

# Escenario 3
#folder = './data/escenario_3/' # Carpeta con los datos
#demanda = 100 # Demanda del cliente
#ctiempo = 1000 # Costo por unidad de tiempo

# Escenario 4
#folder = './data/escenario_4/' # Carpeta con los datos
#demanda = 20000 # Demanda del cliente
#ctiempo = 1000 # Costo por unidad de tiempo

info_acopios = 'centros_acopio.xlsx'
costo_transporte = 'costos_transporte.xlsx'
tiempo_transporte = 'tiempos_transporte.xlsx'
archivos = {
    'info_acopios': info_acopios,
    'costo_transporte' : costo_transporte,
    'tiempo_transporte' : tiempo_transporte,
}
data = model_data(archivos, demanda, ctiempo, folder=folder)
demanda = data['demanda']
ctiempo = data['ctiempo']
params_df = data['params_df']
N, seed, capacidades = model_vars(params_df)

xl = np.zeros(capacidades.shape[0])
xu = capacidades

class Queso(ElementwiseProblem):
    def __init__(self):
        super().__init__(
            n_var=len(xl),
            n_obj=1,
            n_eq_constr=1,
            xl=xl,
            xu=xu
        )
    def _evaluate(self, x, out, *args, **kwargs):
        out['F'] = objective_func(x, N, data)
        individual = np.delete(x, N * 2)
        out['H'] = demanda - np.sum(individual)

class TopOrZeroSampling(Sampling):
    def _do(self, problem, n_samples, **kwargs):
        gen_matrix = np.zeros((n_samples, problem.n_var), dtype=float)
        n_vars = problem.n_var
        for i in range(n_samples):
            indices = np.arange(n_vars - 1)
            np.random.shuffle(indices)
            while np.sum(gen_matrix[i]) < demanda and indices.size > 0:
                idx = indices[0]
                gen_matrix[i, idx] += capacidades[idx]
                indices = np.delete(indices, 0)
                if np.sum(gen_matrix[i]) > demanda:
                    gen_matrix[i, idx] = gen_matrix[i, idx] - (np.sum(gen_matrix[i]) - demanda)
                    break
            gen_matrix[i, N * 2] = np.random.randint(capacidades[N * 2] + 1)
        return gen_matrix

class SinglePointCross(Crossover):
    def __init__(self, prob):
        super().__init__(n_parents=2, n_offsprings=1, prob=prob)
    def _do(self, problem, X, **kwargs):
        _, n_matings, n_var = X.shape
        T = np.zeros((1, n_matings, n_var))
        Y = np.full_like(T, None, dtype=float)
        for idx in range(n_matings):
            p1 = X[0, idx, : n_var//2]
            p2 = X[1, idx, n_var//2 : n_var - 1]
            offspring = np.concatenate((p1, p2))
            if np.sum(offspring) > demanda:
                delta = np.sum(offspring) - demanda
                balance(offspring, capacidades, delta, True)
            else:
                delta = demanda - np.sum(offspring)
                balance(offspring, capacidades, delta, False)
            for i in range(offspring.shape[0]):
                Y[0, idx, i] = offspring[i]
            main = np.random.choice([X[0, idx, n_var-1], X[1, idx, n_var-1]])
            Y[0, idx, n_var-1] = main
        return Y

class ReassignMutation(Mutation):
    def __init__(self, prob):
        super().__init__()
        self.prob = prob
    def _do(self, problem, X, **kwargs):
        for i in range(len(X)):
            r = np.random.random()
            if r < self.prob:
                individual = X[i]
                idx_mut = np.random.randint(individual.shape[0])
                if idx_mut == N*2:
                    X[i, problem.n_var-1] = np.random.randint(capacidades[N * 2] + 1)
                else:
                    if individual[idx_mut] == 0:
                        delta = capacidades[idx_mut]
                        individual[idx_mut] = capacidades[idx_mut]
                        diff = True
                    else:
                        delta = individual[idx_mut]
                        individual[idx_mut] = 0
                        diff = False
                    balance(individual, capacidades, delta, diff)
                    for j in range(individual.shape[0]):
                        X[i, j] = individual[j]
        return X

def run_experiment(p_mutate, p_cross, pop_size, max_gen, block_name, runs=100):
    results = []
    for run in range(runs):
        t_start = timeit.default_timer()
        algorithm = GA(
            pop_size=pop_size,
            sampling=TopOrZeroSampling(),
            crossover=SinglePointCross(prob=p_cross),
            mutation=ReassignMutation(prob=p_mutate),
            eliminate_duplicates=True
        )
        termination = DefaultSingleObjectiveTermination(
            xtol=1e-8,
            cvtol=1e-6,
            ftol=1e-6,
            period=100, 
            n_max_gen=max_gen,
            n_max_evals=100000
        )
        ga = minimize(
            Queso(), 
            algorithm, 
            termination, 
            save_history=False, 
            verbose=False
        )
        t_end = timeit.default_timer()
        t_delta = t_end - t_start
        costo = np.squeeze(ga.F) if ga.F is not None else float('inf')
        results.append({
            'block': block_name,
            'T_mutacion': p_mutate,
            'T_cruce': p_cross,
            'size_pob': pop_size,
            'N_gen': max_gen,
            'run': run + 1,
            'costo': costo,
            'tiempo': t_delta
        })
    return results

def summarize_experiment(results):
    df = pd.DataFrame(results)
    summary = df.groupby(['block', 'T_mutacion', 'T_cruce', 'size_pob', 'N_gen']).agg(
        mean=('costo', 'mean'),
        std=('costo', 'std'),
        avg_time=('tiempo', 'mean')
    ).reset_index()
    summary['var.coeff'] = summary['std'] / summary['mean']
    summary = summary[['block', 'T_mutacion', 'T_cruce', 'size_pob', 'N_gen', 'mean', 'var.coeff', 'avg_time']]
    return summary

block1_p_mutate = [0.5, 0.7, 0.9]
block1_p_cross = [0.5, 0.7, 0.9]
block1_pop_size = [50, 100, 150]
block1_max_gen = [100, 300, 500]

block2_p_mutate = np.arange(0.1, 0.6, 0.1).tolist()
block2_p_cross = np.arange(0.1, 0.8, 0.1).tolist()
block2_pop_size = block1_pop_size
block2_max_gen = block1_max_gen

all_results = []

for p_mutate in block1_p_mutate:
    for p_cross in block1_p_cross:
        for pop_size in block1_pop_size:
            for max_gen in block1_max_gen:
                all_results.extend(run_experiment(p_mutate, p_cross, pop_size, max_gen, 'Bloque 1'))

for p_mutate in block2_p_mutate:
    for p_cross in block2_p_cross:
        for pop_size in block2_pop_size:
            for max_gen in block2_max_gen:
                all_results.extend(run_experiment(p_mutate, p_cross, pop_size, max_gen, 'Bloque 2'))

results_df = pd.DataFrame(all_results)
results_df.to_excel('experiment_results_ga.xlsx', index=False)

summary_df = summarize_experiment(all_results)
summary_df.to_excel('experiment_summary_ga.xlsx', index=False)